In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from IPython.display import Image

import scqubits as scq
import scqubits.utils.sweep_plotting as splot

import qutip as qt

import numpy as np

# Composite Hilbert Spaces, QuTiP Interface

The `HilbertSpace` class provides data structures and methods for handling composite Hilbert spaces which may consist of multiple qubits or qubits and oscillators coupled to each other. To harness the power of QuTiP, a toolbox for studying stationary and dynamical properties of closed and open quantum systems (and much more), `HilbertSpace` provides a convenient interface: it generates `qutip.qobj` objects which are then directly handled by QuTiP.

## Example: two transmons coupled to a harmonic mode

Transmon qubits can be capacitively coupled to a common harmonic mode, realized by an LC oscillator or a transmission-line resonator. The Hamiltonian describing such a composite system is given by:
\begin{equation}
H=H_\text{tmon,1} + H_\text{tmon,2} + \omega_r a^\dagger a + \sum_{j=1,2}g_j n_j(a+a^\dagger),
\end{equation}
where $j=1,2$ enumerates the two transmon qubits, $\omega_r$ is the (angular) frequency of the resonator. Furthermore, $n_j$ is the charge number operator for qubit $j$, and $g_j$ is the coupling strength between qubit $j$ and the resonator.

### Create Hilbert space components

The first step consists of creating the objects describing the individual building blocks of the full Hilbert space. Here, these will be the two transmons and one oscillator:

In [2]:
tmon1 = scq.Transmon(
    EJ=40.0,
    EC=0.2,
    ng=0.3,
    ncut=40,
    truncated_dim=4     # after diagonalization, we will keep 3 levels
)

tmon2 = scq.Transmon(
    EJ=15.0,
    EC=0.15,
    ng=0.0,
    ncut=30,
    truncated_dim=4
)

resonator = scq.Oscillator(
    E_osc=4.5,
    truncated_dim=4  # up to 3 photons (0,1,2,3)
)

The `HilbertSpace` object is then created in one of two ways. The first is by utilizing the GUI, as shown below. Once created, a print call to this object outputs a summary of the composite Hilbert space. The image below is an example of the GUI.

![title](hilbertspace_gui.png)

In [3]:
g = 0.1
hilbertspace_gui = scq.HilbertSpace.create()


Output()

GUI use is optional, of course. The following shows how to achieve the same creation using code.

In [5]:
hilbertspace = scq.HilbertSpace([tmon1, tmon2, resonator])
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

Transmon------------|
                    | EJ: 40.0
                    | EC: 0.2
                    | ng: 0.3
                    | ncut: 40
                    | truncated_dim: 4
                    |
                    | dim: 81


Transmon------------|
                    | EJ: 15.0
                    | EC: 0.15
                    | ng: 0.0
                    | ncut: 30
                    | truncated_dim: 4
                    |
                    | dim: 61


Oscillator----------|
                    | E_osc: 4.5
                    | truncated_dim: 4
                    |
                    | dim: 4




One useful method of the `HilbertSpace` class is `.bare_hamiltonian()`. This yields the bare Hamiltonian of the non-interacting subsystems, expressed as a `qutip.Qobj`:

In [6]:
bare_hamiltonian = hilbertspace.bare_hamiltonian()
bare_hamiltonian

Quantum object: dims = [[4, 4, 4], [4, 4, 4]], shape = (64, 64), type = oper, isherm = True
Qobj data =
[[-48.96753061   0.           0.         ...   0.           0.
    0.        ]
 [  0.         -44.46753061   0.         ...   0.           0.
    0.        ]
 [  0.           0.         -39.96753061 ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...  -9.97661845   0.
    0.        ]
 [  0.           0.           0.         ...   0.          -5.47661845
    0.        ]
 [  0.           0.           0.         ...   0.           0.
   -0.97661845]]

### Set up the interaction between subsystems

There are 3 options that may be utilized when defining an interaction.

#### Option 1: 

A standard interaction term involving multiple subsystems S=1,2,3,... is of the form

$V = g A_1 A_2 A_3 \cdots$

or

$V = g B_1 B_2 B_3 \cdots + \text{h.c.}$

where the operators $A_j$, $B_j$ act on subsystem j, and operators $A_j$ are expected to be Hermitean. 

This structure is captured by using the `add_interaction` method in the most simple case. 

In [4]:
g1 = 0.1  # coupling resonator-tmon1 (without charge matrix elements)
operator1 = tmon1.n_operator()
operator2 = resonator.creation_operator() + resonator.annihilation_operator()

hilbertspace.add_interaction(
    g=g1,
    op1=(operator1, tmon1),
    op2=(operator2, resonator)
)

g2 = 0.2  # coupling resonator-CPB2 (without charge matrix elements)
hilbertspace.add_interaction(
    g=g2,
    op1=tmon2.n_operator,
    op2=resonator.creation_operator,
    add_hc=True
)

NameError: name 'hilbertspace' is not defined

In this instance, op1 and op2 can take a tuple with the operator in the first position and the subsystem in the second position to specify subsystems directly. However, they can also take the operator alone, which will default to the subsystem that the operator is derived from. 

`add_hc = True` will add the hermitian conjugate to the hamiltonian.

#### Option 2:

The `add_interaction` method can also be used to define the interaction directly rather than defaulting to the standard form: 

In [8]:
hilbertspace_example = scq.HilbertSpace([tmon1, tmon2, resonator])

g3 = 0.1
hilbertspace_example.add_interaction(
    expr="g3 * cos(t1) * adag",
    op1=("t1", tmon1.n_operator),
    op2 = ("t2", tmon2.n_operator),
    op3=("adag", resonator.creation_operator(), resonator),
    add_hc=True
)

`expr` is used to define the interaction more explicitly. It should take the form of a python expression, using variables that are already defined as well as variables that are defined in `op1`, `op2`, `op3`, etc. When using `expr`, the `op` arguements should take the form of a tuple with the variable name string in the first position, then the operator in the second, then the optional subsystem in the third. 

Beyond simple python functions, acceptable functions to use in `expr` are as follows:

|  Function             |   Function Translation          |
|-----------------------|---------------------------------|
| `'cos'`               | `'Qobj.cosm'`                   |
| `'sin'`               | `'Qobj.sinm'`                   |
| `'exp'`               | `'Qobj.expm'`                   |
| `'dag'`               | `'Qobj.dag'`                    |
| `'conj'`              | `'Qobj.conj'`                   |
| `'trans'`             | `'Qobj.trans'`                  |
| `'sqrt'`              | `'Qobj.sqrtm'`                  |


#### Option 3:

Finally `add_interaction` can also been used to add a `Qobj` object that has already been properly identity wrapped:

In [9]:
# Generate a Qobj
g = 0.1
a = qt.destroy(4)
kerr = a.dag() * a.dag() * a * a
id = qt.qeye(4)
V = g * qt.tensor(id, id, kerr)

hilbertspace_example.add_interaction(qobj = V)

In this case, the `add_interaction` method only requires the arguement of `qobj` which must be an object of the type `Qobj`. 

With the interactions specified, the full Hamiltonian of the coupled system can be obtained via the method `.hamiltonian()`. Again, this conveniently results in a `qubit.Qobj` operator:

In [10]:
dressed_hamiltonian = hilbertspace.hamiltonian()
dressed_hamiltonian

Quantum object: dims = [[4, 4, 4], [4, 4, 4]], shape = (64, 64), type = oper, isherm = True
Qobj data =
[[-4.89675306e+01  3.00000000e-02  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 3.00000000e-02 -4.44675306e+01  4.24264069e-02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  4.24264069e-02 -3.99675306e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -9.97661845e+00
   4.24264070e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  4.24264070e-02
  -5.47661845e+00  5.19615243e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   5.19615243e-02 -9.76618450e-01]]

### Obtaining the eigenspectrum via QuTiP

Since the Hamiltonian obtained this way is a proper `qutip.qobj`, all QuTiP routines are now available. In the first case, we are still making use of the scqubit `HilbertSpace.eigensys()` method. In the second, case, we use QuTiP's method `.eigenenergies()`:

In [11]:
evals, evecs = hilbertspace.eigensys(evals_count=4)
print(evals)

[-48.97770317 -45.02707241 -44.36656205 -41.18438832]


In [12]:
dressed_hamiltonian = hilbertspace.hamiltonian()
dressed_hamiltonian.eigenenergies()

array([-48.97770317, -45.02707241, -44.36656205, -41.18438832,
       -41.1776098 , -40.46448065, -39.76202396, -37.45533167,
       -37.22705488, -36.65156212, -36.56694139, -35.88617024,
       -35.14255357, -33.58960343, -33.38439485, -33.12061816,
       -32.66494366, -32.04065582, -31.96284558, -30.93536847,
       -29.65538466, -29.63912664, -28.97946785, -28.85287223,
       -28.64748897, -28.09427075, -27.35745603, -26.97461415,
       -26.21586637, -25.79648462, -25.3216198 , -25.07747135,
       -24.37567532, -24.24760531, -23.66618527, -23.15199748,
       -22.2580349 , -22.06752989, -21.57303944, -21.26626732,
       -20.85288717, -20.51511596, -19.78521899, -19.1938092 ,
       -18.41228666, -17.73463904, -17.66080329, -16.93618597,
       -16.66699841, -15.89010361, -15.58181323, -14.68288545,
       -13.84725844, -13.27037775, -13.08388811, -12.34366698,
       -11.62655644, -10.308913  ,  -9.23168892,  -8.32791939,
        -8.13883089,  -5.82301882,  -4.18180351,  -0.88